In [1]:
import numpy as np
import scipy.stats
from scipy.optimize import minimize

In [2]:
data = np.genfromtxt('wang.csv', delimiter=",")

V = data[:,:3]
U1 = data[:,3:6]
U2 = data[:,6:]

In [3]:
def closure(mat):
    mat = np.atleast_2d(mat)
    mat = mat / mat.sum(axis=1, keepdims=True)
    return mat.squeeze()

In [4]:
def _gram_schmidt_basis(n):
    basis = np.zeros((n, n-1))
    for j in range(n-1):
        i = j + 1
        e = np.array([(1/i)]*i + [-1] +
                     [0]*(n-i-1))*np.sqrt(i/(i+1))
        basis[:, j] = e
    return basis.T

In [5]:
def clr(mat):
    mat = closure(mat)
    lmat = np.log(mat)
    gm = lmat.mean(axis=-1, keepdims=True)
    return (lmat - gm).squeeze()

In [6]:
def clr_inv(mat):
    return closure(np.exp(mat))

In [7]:
def ilr(mat):
    d = mat.shape[1]
    W = _gram_schmidt_basis(d)
    return clr(mat) @ W.T

In [8]:
def ilr_inv(mat):
    d = mat.shape[1] + 1
    W = _gram_schmidt_basis(d)
    return clr_inv(mat @ W)

In [9]:
def centralize(mat):
    mat = closure(mat)
    cen = scipy.stats.gmean(mat, axis=0)
    return clr_inv(clr(mat)- clr(cen))

In [10]:
def KL_coeff(V, U_vec):
    k = len(U_vec)
    V_tilde = np.exp(clr(V))
    C_vec = [clr(U) for U in U_vec]
    
    def kl_loss(beta):
        # beta in R^k
        ilr_estimator = sum(beta[i] * C_vec[i] for i in range(k))
        h = sum(beta[i] * np.trace(V_tilde.T @ C_vec[i]) for i in range(k))
        return np.sum(np.exp(ilr_estimator)) - h
    
    def kl_loss_grad(beta):
        ilr_estimator = sum(beta[i] * C_vec[i] for i in range(k))
        return np.array([np.sum(C_vec[i] * np.exp(ilr_estimator)) - np.trace(V_tilde.T@C_vec[i]) for i in range(k)])
    
    #print(kl_loss([-0.1,0.2,-0.3,0.4]))
    
    return minimize(kl_loss, np.array([0.0]*k), method='BFGS', jac=kl_loss_grad, tol=1e-16, options={'gtol': 1e-012, 'disp':True}).x

In [11]:
KL_coeff(V, [U1,U2])

Optimization terminated successfully.
         Current function value: 47.066758
         Iterations: 9
         Function evaluations: 10
         Gradient evaluations: 10


array([0.8779684 , 0.08993897])

In [12]:
def L2_coeff(V, U_vec):
    k = len(U_vec)
    Y = ilr(V)
    X_vec = [ilr(U) for U in U_vec]
    A = np.array([ [np.trace(X_vec[i].T @ X_vec[j]) for i in range(k)] for j in range(k)])
    b = np.array( [np.trace(X_vec[i].T@Y) for i in range(k)]).T
    return np.linalg.solve(A,b)

In [13]:
L2_coeff(V, [U1,U2])

array([0.87427064, 0.0795275 ])

In [14]:
beta_KL = KL_coeff(V, [U1,U2])

Optimization terminated successfully.
         Current function value: 47.066758
         Iterations: 9
         Function evaluations: 10
         Gradient evaluations: 10


In [15]:
def fitted_outputs(beta, U_vec, V_mean):
    return clr_inv( sum(beta[i]* clr(U_vec[i]) for i in range(len(U_vec))) + clr(V_mean))

In [16]:
V_mean = np.array([0.01275, 0.47401, 0.51324] * 16).reshape((16,3))

In [17]:
V_KL_fit = fitted_outputs(KL_coeff(V, [U1,U2]), [U1,U2], V_mean)

Optimization terminated successfully.
         Current function value: 47.066758
         Iterations: 9
         Function evaluations: 10
         Gradient evaluations: 10


In [18]:
V_KL_fit

array([[0.01594618, 0.58222161, 0.40183221],
       [0.01946744, 0.57279715, 0.40773541],
       [0.01931529, 0.5451404 , 0.43554431],
       [0.01808075, 0.51478066, 0.46713859],
       [0.01804561, 0.5135289 , 0.46842549],
       [0.01714254, 0.48803445, 0.49482301],
       [0.01781135, 0.44697976, 0.53520889],
       [0.01597392, 0.4421766 , 0.54184948],
       [0.01329568, 0.44651637, 0.54018795],
       [0.00990434, 0.48449109, 0.50560457],
       [0.0098485 , 0.44981326, 0.54033824],
       [0.00939282, 0.44107229, 0.54953489],
       [0.00848634, 0.43539626, 0.55611741],
       [0.00763713, 0.4225101 , 0.56985277],
       [0.00837836, 0.39577566, 0.59584598],
       [0.00682803, 0.39622273, 0.59694924]])

In [19]:
V_L2_fit = fitted_outputs(L2_coeff(V, [U1,U2]), [U1,U2], V_mean)

In [20]:
V_L2_fit

array([[0.01587625, 0.58166976, 0.40245399],
       [0.01935385, 0.57214031, 0.40850584],
       [0.01933004, 0.54456283, 0.43610713],
       [0.01818894, 0.51419253, 0.46761854],
       [0.01798725, 0.51305528, 0.46895746],
       [0.01708085, 0.48788557, 0.49503358],
       [0.01771023, 0.44688781, 0.53540197],
       [0.01594622, 0.44207478, 0.54197901],
       [0.01332406, 0.44646983, 0.54020611],
       [0.01002833, 0.48414628, 0.50582539],
       [0.00988742, 0.45020903, 0.53990355],
       [0.00935165, 0.44139562, 0.54925272],
       [0.00850669, 0.43584173, 0.55565158],
       [0.00766174, 0.42323713, 0.56910114],
       [0.00839189, 0.39677798, 0.59483012],
       [0.00681721, 0.39696447, 0.59621832]])

In [21]:
U1_mean = np.array([0.08077, 0.44037, 0.47887] * 16).reshape((16,3))

In [22]:
U2_mean = np.array([0.00240, 0.28059, 0.71701] * 16).reshape((16,3))

In [23]:
V_obs = clr_inv(clr(V) + clr(V_mean))

In [24]:
V_obs

array([[0.02399536, 0.56800055, 0.40800409],
       [0.02299571, 0.53999987, 0.43700442],
       [0.02099602, 0.51599776, 0.46300622],
       [0.01899663, 0.49300361, 0.48799976],
       [0.01799652, 0.47399705, 0.50800643],
       [0.01599701, 0.46299468, 0.52100831],
       [0.01499762, 0.46100326, 0.52399913],
       [0.01399735, 0.45699544, 0.5290072 ],
       [0.01199779, 0.47899855, 0.50900365],
       [0.00999807, 0.48200217, 0.50799975],
       [0.00999808, 0.4740046 , 0.51599732],
       [0.00899819, 0.47000298, 0.52099883],
       [0.00799842, 0.44600168, 0.5459999 ],
       [0.00799866, 0.43199708, 0.56000426],
       [0.00699876, 0.39899424, 0.594007  ],
       [0.0069986 , 0.41999761, 0.5730038 ]])

In [25]:
U1_obs = clr_inv(clr(U1)+clr(U1_mean))

In [26]:
U2_obs = clr_inv(clr(U2)+clr(U2_mean))

In [27]:
scipy.stats.gmean(V_obs, axis=0)

array([0.01268569, 0.471622  , 0.51065668])

In [28]:
def L1_error(V, V_):
    return np.sum(np.absolute(V - V_))

In [29]:
L1_error(V_obs, V_KL_fit)

0.656814551159535

In [30]:
L1_error(V_obs, V_L2_fit)

0.6430739843417799

In [31]:
def L2_error(V, V_):
    return np.sum(np.square(V - V_))

In [32]:
L2_error(V_obs, V_KL_fit)

0.01635535787366693

In [33]:
L2_error(V_obs, V_L2_fit)

0.015857011059149634

In [34]:
def Fisher_error(V, V_):
    return 2* np.trace(np.arccos(np.sqrt(V)@np.sqrt(V_.T)))

In [35]:
Fisher_error(V_obs, V_KL_fit)

0.7137144338010832

In [36]:
Fisher_error(V_obs, V_L2_fit)

0.7025295433044868

In [37]:
def symKL_error(V, V_):
    return np.sum((V-V_) * np.log(np.divide(V, V_)))

In [38]:
symKL_error(V_obs, V_KL_fit)

0.038522274611487405

In [39]:
symKL_error(V_obs, V_L2_fit)

0.03754442482556374

# D17 Aitchison

In [40]:
# Dataset 17 Aitchison (with V[13] adjusted due to error in book)
V_obs = [
[0.27,0.28,0.45],
[0.02,0.03,0.95],
[0.12,0.16,0.72],
[0.83,0.02,0.15],
[0.24,0.22,0.54],
[0.16,0.20,0.64],
[0.31,0.08,0.61],
[0.05,0.85,0.10],
[0.06,0.06,0.88],
[0.08,0.31,0.61],
[0.18,0.20,0.62],
[0.17,0.19,0.64],
[0.04,0.17,0.79],
[0.08,0.25,0.67],
[0.11,0.34,0.55] ]

U1_obs = [
[0.70,0.07,0.23],
[0.19,0.16,0.65],
[0.18,0.26,0.54],
[0.02,0.02,0.96],
[0.08,0.16,0.76],
[0.14,0.18,0.68],
[0.16,0.11,0.73],
[0.04,0.06,0.90],
[0.06,0.54,0.40],
[0.12,0.22,0.66],
[0.06,0.02,0.92],
[0.16,0.04,0.80],
[0.27,0.17,0.56],
[0.21,0.51,0.28],
[0.15,0.15,0.70] ]

In [41]:
V = centralize(V_obs)

In [42]:
V_mean = np.array([scipy.stats.gmean(V_obs, axis=0)]*15)

In [43]:
U1 = centralize(U1_obs)

In [44]:
U1_mean = np.array([scipy.stats.gmean(U1_obs, axis=0)]*15)

In [45]:
V_KL_fit = fitted_outputs(KL_coeff(V, [U1]), [U1], V_mean)

Optimization terminated successfully.
         Current function value: 42.089996
         Iterations: 5
         Function evaluations: 7
         Gradient evaluations: 7


In [46]:
V_KL_fit

array([[0.03951574, 0.18215582, 0.77832843],
       [0.12903745, 0.181579  , 0.68938354],
       [0.128372  , 0.1337589 , 0.73786911],
       [0.2852205 , 0.36472724, 0.35005226],
       [0.2043195 , 0.17760618, 0.61807432],
       [0.15368265, 0.17085643, 0.67546092],
       [0.14032134, 0.22107094, 0.63860772],
       [0.25694805, 0.26216224, 0.48088971],
       [0.19759914, 0.07433079, 0.72807007],
       [0.16628862, 0.15172414, 0.68198724],
       [0.17621713, 0.4154679 , 0.40831496],
       [0.12357658, 0.34198108, 0.53444234],
       [0.10294354, 0.17032293, 0.72673353],
       [0.09251495, 0.07217771, 0.83530733],
       [0.1476464 , 0.18880362, 0.66354998]])

In [47]:
V_L2_fit = fitted_outputs(L2_coeff(V, [U1]), [U1], V_mean)

In [48]:
V_L2_fit

array([[0.09792262, 0.19472225, 0.70735513],
       [0.14310575, 0.19080841, 0.66608584],
       [0.14335716, 0.17274296, 0.68389988],
       [0.19504864, 0.25181559, 0.55313577],
       [0.16732023, 0.18967804, 0.64300173],
       [0.15174759, 0.1868903 , 0.66136211],
       [0.14717089, 0.20383938, 0.64898974],
       [0.18296573, 0.21891523, 0.59811904],
       [0.16773263, 0.14339967, 0.6888677 ],
       [0.15600298, 0.17975888, 0.66423814],
       [0.16395125, 0.26011114, 0.57593761],
       [0.14267054, 0.23889999, 0.61842947],
       [0.13296616, 0.18722809, 0.67980575],
       [0.13076145, 0.14293229, 0.72630626],
       [0.14965188, 0.19320655, 0.65714157]])

In [49]:
L1_error(V_obs, V_KL_fit)

6.20191411403734

In [50]:
L1_error(V_obs, V_L2_fit)

5.8500769095778855

In [51]:
L2_error(V_obs, V_KL_fit)

1.6360709884415294

In [52]:
L2_error(V_obs, V_L2_fit)

1.7341980690791383

In [53]:
Fisher_error(V_obs, V_KL_fit)

7.744428875681296

In [54]:
Fisher_error(V_obs, V_L2_fit)

7.069610065046507

In [55]:
symKL_error(V_obs, V_KL_fit)

6.116991096034899

In [56]:
symKL_error(V_obs, V_L2_fit)

5.939364536691749

In [57]:
KL_coeff(V, [U1])

Optimization terminated successfully.
         Current function value: 42.089996
         Iterations: 5
         Function evaluations: 7
         Gradient evaluations: 7


array([-0.55688544])

In [58]:
L2_coeff(V, [U1])

array([-0.18759286])

# Artificial  Dataset

In [59]:
np.random.seed(100)

In [60]:
n, d, k = 20, 10, 4

In [61]:
U_obs = [ ilr_inv(np.random.randn(n,d-1)) for i in range(k)]

In [62]:
beta_true = [(-1)**j * 0.1 * k for j in range(1,k+1)]

In [63]:
V_obs = ilr_inv(sum( beta_true[i] * ilr(U_obs[i]) for i in range(k)) + 0.2* np.random.randn(n,d-1) )

In [64]:
V = centralize(V_obs)

In [65]:
V_mean = np.array([scipy.stats.gmean(V_obs, axis=0)]*n)

In [66]:
U = [centralize(U_obs[i]) for i in range(k)]

In [67]:
V_KL_fit = fitted_outputs(KL_coeff(V, U), U, V_mean)
V_L2_fit = fitted_outputs(L2_coeff(V, U), U, V_mean)

         Current function value: 128.486627
         Iterations: 14
         Function evaluations: 99
         Gradient evaluations: 86


In [68]:
L1_error(V_obs, V_KL_fit)

2.558752259687211

In [69]:
L1_error(V_obs, V_L2_fit)

2.560728436069907

In [70]:
L2_error(V_obs, V_KL_fit)

0.0695671209677328

In [71]:
L2_error(V_obs, V_L2_fit)

0.07033931601600182

In [72]:
Fisher_error(V_obs, V_KL_fit)

3.2072274136611716

In [73]:
Fisher_error(V_obs, V_L2_fit)

3.203967053678765

In [74]:
symKL_error(V_obs, V_KL_fit)

0.559796441341691

In [75]:
symKL_error(V_obs, V_L2_fit)

0.5628690010951222

In [76]:
KL_coeff(V, U)

         Current function value: 128.486627
         Iterations: 14
         Function evaluations: 99
         Gradient evaluations: 86


array([-0.41220933,  0.3948095 , -0.41851323,  0.39962675])

In [77]:
L2_coeff(V, U)

array([-0.41981132,  0.38955447, -0.42349373,  0.40064569])